# Исследование сервиса для чтения книг по подписке

**Цель** - проанализировать базу данных

**Задача** -  сформулировать ценностное предложение для нового продукта

**Описание данных:**

Таблица books содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

Таблица ratings содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

**Ход исследования:**
1. загрузка и чтение основной информации датасетов
2. изучение данных:
- сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
3. составление выводов
4. формулировка рекомендаций

## Загрузка и чтение основной информации датасетов

### Подготовка к чтению

In [1]:
# импорт библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# установка параметров
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
con=engine.connect()

In [5]:
#создание функции для отображение sql_запроса
def sql_query(query):
    result = pd.io.sql.read_sql(query, con)
    return result

### Таблица books

In [6]:
books = '''
            SELECT *
            FROM books
            LIMIT 5;
        '''
sql_query(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
books_count = '''
            SELECT
                COUNT (*) 
            FROM books;
            '''
sql_query(books_count)

,count
0,1000


In [8]:
books_info = '''
            SELECT
                COUNT (DISTINCT book_id) AS books_unique,
                COUNT (DISTINCT author_id) AS author_unique,
                COUNT (DISTINCT title) AS title_unique,
                MIN (num_pages) AS pages_min,
                MAX (num_pages) AS pages_max,
                MIN (publication_date) AS date_min,
                MAX (publication_date) AS date_max,
                COUNT (DISTINCT publisher_id) AS publisher_unique
            FROM books;
            '''
sql_query(books_info)

,books_unique,author_unique,title_unique,pages_min,pages_max,date_min,date_max,publisher_unique
0,1000,636,999,14,2690,1952-12-01,2020-03-31,340


In [9]:
duplicated_title = '''  SELECT *
                              FROM books 
                              WHERE books.title = (SELECT 
                                                       title
                                                   FROM books
                                                   GROUP BY title
                                                   HAVING COUNT(*)>1)
                        '''
sql_query(duplicated_title)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


In [10]:
books_null = '''
            SELECT *
            FROM books
            WHERE (author_id IS NULL) OR (num_pages IS NULL) OR (publication_date IS NULL) OR (publisher_id IS NULL);
            '''
sql_query(books_null)

,book_id,author_id,title,num_pages,publication_date,publisher_id


В таблице books представлены данные о книгах сервиса. Всего в датасете содержится 1000 книг, опубликованных в период с 1-12-1952 по 31-03-2020. Название одной книги повторяется, однако это не считается дубликатом, т.к. количество страниц, издательство и дата выпуска книг отличаются. Возможно, это разные версии книги, отличающиеся по переводу. Пропусков в таблице нет

### Таблица authors

In [11]:
authors = '''
            SELECT *
            FROM authors
            LIMIT 5;
        '''
sql_query(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [12]:
authors_count = '''
            SELECT
                COUNT (*) 
            FROM authors;
            '''
sql_query(authors_count)

,count
0,636


In [13]:
authors_info = '''
                SELECT
                    COUNT (DISTINCT author_id) AS author_id_unique,
                    COUNT (DISTINCT author) AS author_unique
                FROM authors;
                '''
sql_query(authors_info)

,author_id_unique,author_unique
0,636,636


Таблица authors содержит в себе данные об авторах книг, а именно идентификатор и имя автора. Всего авторов - 636. Пропусков в таблице нет.

### Таблица ratings

In [14]:
ratings = '''
            SELECT *
            FROM ratings
            LIMIT 5;
        '''
sql_query(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
ratings_count = '''
            SELECT
                COUNT (*) 
            FROM ratings;
            '''
sql_query(ratings_count)

,count
0,6456


In [16]:
ratings_info = '''
                 SELECT 
                       COUNT(DISTINCT rating_id) AS rating_unique,
                       COUNT(DISTINCT book_id) AS book_unique,
                       COUNT(DISTINCT username) AS user_unique,
                       MIN(rating) AS rating_min,
                       MAX(rating) AS rating_max
                 FROM ratings;
              '''
sql_query(ratings_info)

,rating_unique,book_unique,user_unique,rating_min,rating_max
0,6456,1000,160,1,5


In [17]:
ratings_null = '''
                SELECT *
                FROM ratings
                WHERE (book_id IS NULL) OR (username IS NULL) OR (rating IS NULL);
                '''
sql_query(ratings_null)

,rating_id,book_id,username,rating


Таблица ratings содержит в себе данные о рейтингах книг. Всего оценило книги 160 пользователей. Значение рейтинга варьируется от 1 до 5. Пропусков в таблице нет.

### Таблица reviews

In [18]:
reviews = '''
            SELECT *
            FROM reviews
            LIMIT 5;
        '''
sql_query(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [19]:
reviews_count = '''
                SELECT
                     COUNT (*) 
                FROM reviews;
            '''
sql_query(reviews_count)

,count
0,2793


In [20]:
reviews_info = '''
                 SELECT 
                       COUNT(DISTINCT review_id) AS reviews_unique,
                       COUNT(DISTINCT book_id) AS book_unique,
                       COUNT(DISTINCT username) AS user_unique,
                       COUNT(DISTINCT text) AS text_unique
                 FROM reviews;
              '''
sql_query(reviews_info)

,reviews_unique,book_unique,user_unique,text_unique
0,2793,994,160,2793


In [21]:
reviews_null = '''
                SELECT *
                FROM reviews
                WHERE (book_id IS NULL) OR (username IS NULL);
                '''
sql_query(reviews_null)

,review_id,book_id,username,text


Таблица reviews содержит в себе данные обзоров на книги. Обзоры на книги оставили 160 пользователей, написали 2793 текстовых отзывов. Пропусков в таблице нет.

### Таблица publishers

In [22]:
publishers = '''
               SELECT *
               FROM publishers
               LIMIT 5;
             '''
sql_query(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [23]:
publishers_len = '''
                SELECT
                     COUNT (*) 
                FROM publishers;
              '''
sql_query(publishers_len)

,count
0,340


In [24]:
publishers_info = '''
                 SELECT 
                       COUNT(DISTINCT publisher_id) AS publ_id_unique,
                       COUNT(DISTINCT publisher) AS publisher_unique
                 FROM publishers;
              '''
sql_query(publishers_info)

,publ_id_unique,publisher_unique
0,340,340


Таблица publishers содержит в себе данные об издательствах. Всего издательств - 340. Пропусков в таблице нет.

Общая информация о датасете:
- 1000 книг
- 636 авторов
- 160 пользователей оставили текстовый обзор и поставили оценку от 1 до 5
- 340 издательств выпустили книги датасета.

## Изучение данных

### Количество книг, вышедших после 1 января 2000

In [25]:
books_after_2000 = '''
                     SELECT 
                           COUNT(book_id) AS books_after_2000_count
                     FROM books
                     WHERE publication_date >= '2000-01-01';
                  '''
sql_query(books_after_2000)

,books_after_2000_count
0,821


После 1 января 2020 года было выпущено 821 книга

### Количество обзоров и средняя оценка книг

In [33]:
review_rating = '''
                   SELECT 
                         b.book_id,
                         b.title,
                         COUNT( DISTINCT rev.review_id) AS review_count,
                         AVG (rat.rating) AS avg_rating
                   FROM books AS b
                   LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
                   LEFT JOIN ratings AS rat On b.book_id=rat.book_id
                   GROUP BY b.book_id
                   ORDER BY review_count DESC
                   LIMIT 15;
                '''
sql_query(review_rating)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,207,Eat Pray Love,6,3.395833
3,656,The Book Thief,6,4.264151
4,733,The Giver (The Giver #1),6,3.750000
5,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,627,The Alchemist,6,3.789474


In [27]:
rating_review = '''
                   SELECT 
                         b.book_id,
                         b.title,
                         COUNT( DISTINCT rev.review_id) AS review_count,
                         AVG (rat.rating) AS avg_rating
                   FROM books AS b
                   LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
                   LEFT JOIN ratings AS rat On b.book_id=rat.book_id
                   GROUP BY b.book_id
                   ORDER BY avg_rating DESC
                   LIMIT 15;
                '''
sql_query(rating_review)

,book_id,title,review_count,avg_rating
0,182,Dead Souls,2,5.0
1,275,Geek Love,2,5.0
2,169,Crucial Conversations: Tools for Talking When ...,2,5.0
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
4,229,Evil Under the Sun (Hercule Poirot #24),2,5.0
5,224,Evening Class,2,5.0
6,57,Act of Treason (Mitch Rapp #9),2,5.0
7,20,A Fistful of Charms (The Hollows #4),2,5.0
8,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
9,76,Angels Fall,2,5.0


Самое большое количество обзоров оставили книгу Twilight 1 (Сумерки часть 1) - 7 обзоров.

В датасете имеются книги с средним рейтингом 5.0, но чаще всего на такие книги мало обзоров.

### Издательство, выпустившее объемные книги

In [28]:
publisher_books = '''
                     SELECT 
                           p.publisher,
                           COUNT(b.book_id) AS books_count
                     FROM publishers AS p
                     JOIN books AS b ON p.publisher_id=b.publisher_id
                     WHERE b.num_pages > 50
                     GROUP by p.publisher_id
                     ORDER BY COUNT(b.book_id) DESC
                     LIMIT 1;
                  '''
sql_query(publisher_books)

,publisher,books_count
0,Penguin Books,42


Издательством, выпустившим наибольшее количество книг (42 книги) толще 50 листов, является издательство Penguin Books.

### Автор с самой высокой средней оценкой книг

In [29]:
top_author = '''
                SELECT
                     a.author,
                     AVG(r.rating) AS avg_rating
                FROM books b
                LEFT JOIN authors AS a ON b.author_id=a.author_id
                LEFT JOIN ratings AS r ON r.book_id=b.book_id
                GROUP BY a.author
                HAVING COUNT(r.rating) > 50
                ORDER BY avg_rating DESC
                LIMIT 1; 
             '''
sql_query(top_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


Автором с самой высокой средней оценкой книг, которые набрали более 50 оценок является J.K. Rowling с иллюстратором Mary GrandPré.

### Cреднее количество обзоров от активных пользователей

In [30]:
review_topuser = '''WITH
                    rating_count AS (SELECT
                                           username
                                     FROM ratings
                                     GROUP BY username
                                     HAVING COUNT(ratings) > 48
                                     ),
                    count_review AS (SELECT
                                           reviews.username,
                                           COUNT(review_id) AS count_reviews
                                     FROM reviews
                                     RIGHT JOIN rating_count AS rc ON rc.username=reviews.username
                                     GROUP BY reviews.username)
                                     
                    SELECT 
                          AVG(count_reviews) AS avg_review
                    FROM count_review;
                 '''
sql_query(review_topuser)

,avg_review
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора.

## Выводы:
1. данные открыты и прочтены
2. данные подробно изучены:
- После 1 января 2020 года было выпущено 821 книга.
- Самое большое количество обзоров оставили книгу Twilight 1 (Сумерки часть 1) - 7 обзоров.
- В датасете имеются книги с средним рейтингом 5.0, но чаще всего на такие книги мало обзоров.
- Издательством, выпустившим наибольшее количество книг (42 книги) толще 50 листов, является издательство Penguin Books.
- Автором с самой высокой средней оценкой книг, которые набрали более 50 оценок является J.K. Rowling с иллюстратором Mary GrandPré.
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора.
3. составлен вывод
4. даны рекомендации

## Рекомендации

Компании стоит обратить внимание на продвижение продаж бестселлеров, а также книг с высоким рейтингом.

В качестве нового продукта сервиса можно внедрить раздел аудиокниг, начать который стоит также с бестселлеров или классической отечественной и зарубежной литературы.

Для увеличения продаж и активности сервиса стоит проводить рекламные акции по выдачи, например, подарочных промокодов при написании отзыва и добавлении оценки, а также проводить специальные предложения для особо активных клиентов.